In [3]:
import gzip
import csv
import numpy as np
import urllib
import os
import tensorflow as tf

In [2]:
URL = 'http://ai.stanford.edu/~btaskar/ocr/letter.data.gz'

In [4]:
urllib.request.urlretrieve(URL, filename='letter.data.gz')

('letter.data.gz', <http.client.HTTPMessage at 0x26ccd978128>)

In [5]:
filename = 'letter.data.gz'
file_ = gzip.open(filename, 'rt') 

In [6]:
reader=csv.reader(file_, delimiter='\t')

In [7]:
lines = list(reader)

In [9]:
np.shape(lines)

(52152, 135)

In [10]:
lines = sorted(lines, key=lambda x: int(x[0]))

In [ ]:
# id, target letter, pixel, id of following letter 

In [29]:
data,target = [],[]
next_=None
for line in lines:
    if not next_:
        data.append([])
        target.append([])
    else:
        assert next_==int(line[0])
        
    next_=int(line[2]) if int(line[2])>-1 else None
    pixels = np.array([int(x) for x in line[6:134]])
    pixels = np.reshape(pixels,[16,8])
    data[-1].append(pixels)
    target[-1].append(line[1])

In [37]:
max_length=max(len(x) for x in target)

In [42]:
[np.zeros((3,2))]*2

[array([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]]), array([[ 0.,  0.],
        [ 0.,  0.],
        [ 0.,  0.]])]

In [43]:
padding = np.zeros((16,8))
data = [x + ([padding]*(max_length-len(x))) for x in data]
target = [x+(['']*(max_length-len(x))) for x in target]

In [47]:
data = np.array(data)
target= np.array(target)

In [50]:
np.shape(target)

(6877, 14)

In [72]:
class SequenceLabellingModel:

    def __init__(self, data, target, params):
        self.data = data
        self.target = target
        self.params = params
        self.prediction
        self.cost
        self.error
        self.optimize

 
    def length(self):
        used = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
        length = tf.reduce_sum(used, reduction_indices=1)
        length = tf.cast(length, tf.int32)
        return length

  
    def prediction(self):
        output, _ = tf.nn.dynamic_rnn(
            tf.nn.rnn_cell.GRUCell(self.params.rnn_hidden),
            self.data,
            dtype=tf.float32,
            sequence_length=self.length,
        )
        # Softmax layer.
        max_length = int(self.target.get_shape()[1])
        num_classes = int(self.target.get_shape()[2])
        weight = tf.Variable(tf.truncated_normal(
            [self.params.rnn_hidden, num_classes], stddev=0.01))
        bias = tf.Variable(tf.constant(0.1, shape=[num_classes]))
        # Flatten to apply same weights to all time steps.
        output = tf.reshape(output, [-1, self.params.rnn_hidden])
        prediction = tf.nn.softmax(tf.matmul(output, weight) + bias)
        prediction = tf.reshape(prediction, [-1, max_length, num_classes])
        return prediction

  
    def cost(self):
        # Compute cross entropy for each frame.
        cross_entropy = self.target * tf.log(self.prediction)
        cross_entropy = -tf.reduce_sum(cross_entropy, reduction_indices=2)
        mask = tf.sign(tf.reduce_max(tf.abs(self.target), reduction_indices=2))
        cross_entropy *= mask
        # Average over actual sequence lengths.
        cross_entropy = tf.reduce_sum(cross_entropy, reduction_indices=1)
        cross_entropy /= tf.cast(self.length, tf.float32)
        return tf.reduce_mean(cross_entropy)

 
    def error(self):
        mistakes = tf.not_equal(
            tf.argmax(self.target, 2), tf.argmax(self.prediction, 2))
        mistakes = tf.cast(mistakes, tf.float32)
        mask = tf.sign(tf.reduce_max(tf.abs(self.target), reduction_indices=2))
        mistakes *= mask
        # Average over actual sequence lengths.
        mistakes = tf.reduce_sum(mistakes, reduction_indices=1)
        mistakes /= tf.cast(self.length, tf.float32)
        return tf.reduce_mean(mistakes)


    def optimize(self):
        gradient = self.params.optimizer.compute_gradients(self.cost)
        try:
            limit = self.params.gradient_clipping
            gradient = [
                (tf.clip_by_value(g, -limit, limit), v)
                if g is not None else (None, v)
                for g, v in gradient]
        except AttributeError:
            print('No gradient clipping parameter specified.')
        optimize = self.params.optimizer.apply_gradients(gradient)
        return optimize


In [54]:
import attrdict
import random

In [55]:
params = attrdict.AttrDict(
rnn_cell=tf.nn.rnn_cell.GRUCell,
rnn_hidden=300,
optimizer=tf.train.RMSPropOptimizer(0.002),
gradient_clipping=5,
batch_size=10,
epochs=20,
epoch_size=50,
)

In [58]:
data = data.reshape(data.shape[:2]+(-1,))

In [59]:
target_onehot = np.zeros(target.shape+(26,))

In [60]:
for index, letter in np.ndenumerate(target):
    if letter:
        target_onehot[index][ord(letter)-ord('a')]=1
        

In [62]:
order = np.random.permutation(len(data))

In [63]:
data=data[order]

In [64]:
target_onehot = target_onehot[order] 

In [75]:
data.shape

(6877, 14, 128)

In [77]:
target_onehot.shape

(6877, 14, 26)

In [78]:
def batched(data, target, batch_size):
    """
    
    """
    epoch = 0
    offset = 0
    while True:
        old_offset = offset
        offset = (offset + batch_size) % (target.shape[0] - batch_size)

        if offset < old_offset:
            # New epoch, need to shuffle data
            p = np.random.permutation(len(data))
            data = data[p]
            target = target[p]
            epoch += 1

        batch_data = data[offset:(offset + batch_size), :]
        batch_target = target[offset:(offset + batch_size), :]
        yield batch_data, batch_target, epoch
        

In [65]:
_, length, image_size = data.shape

In [68]:
num_classes = target_onehot.shape[2]

In [73]:
train_data = tf.placeholder(tf.float32, [None, length, image_size])
train_target = tf.placeholder(tf.float32, [None, length, num_classes])
model = SequenceLabellingModel(train_data, train_target, params)

In [79]:
batches = batched(data, target_onehot, params.batch_size)

In [80]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

for index, batch in enumerate(batches):
    batch_data = batch[0]
    batch_target = batch[1]
    epoch = batch[2]
    if epoch >= params.epochs:
        break
    feed = {train_data: batch_data, train_target: batch_target}
    error, _ = sess.run([model.error, model.optimize], feed)
    print('{}: {:3.6f}%'.format(index + 1, 100 * error))

TypeError: Fetch argument <bound method SequenceLabellingModel.error of <__main__.SequenceLabellingModel object at 0x0000026CD398C240>> has invalid type <class 'method'>, must be a string or Tensor. (Can not convert a method into a Tensor or Operation.)